In [ ]:

#TODO! \
    #REF https://developers.google.com/maps/documentation/places/web-service/place-types?hl=ko&_gl=1*dfyfk*_up*MQ..*_ga*MTA5OTg2NTM5OS4xNzYyODMzOTgw*_ga_SM8HXJ53K2*czE3NjI4MzM5NzkkbzEkZzAkdDE3NjI4MzM5NzkkajYwJGwwJGgw*_ga_NRWSTWS78N*czE3NjI4MzM5ODMkbzEkZzEkdDE3NjI4MzYxMDIkajEkbDAkaDA.
import requests
import json

import os
# 환경 변수에 저장했다고 가정 (예: .env 파일)
API_KEY = os.environ.get("GOOGLE_PLACES_API_KEY")

def nearby_search_places(api_key: str, latitude: float, longitude: float, radius: int, place_type: str, keyword: str = None) -> list:
    """
    Google Places API의 Nearby Search를 사용하여 주변 장소를 검색합니다.

    Args:
        api_key: Google Places API 키.
        latitude: 검색 중심점의 위도 (Lat).
        longitude: 검색 중심점의 경도 (Lng).
        radius: 검색 반경 (미터 단위). (최대 50000m)
        place_type: 검색할 장소의 유형 (예: 'restaurant', 'tourist_attraction').
        keyword: 추가적인 키워드 (예: '파스타').
        
    Returns:
        주변 장소 목록 (이름, 주소, 위도, 경도, Place ID)
    """
    
    BASE_URL = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    params = {
        'key': api_key,
        'location': f"{latitude},{longitude}",
        'radius': radius,
        'type': place_type,
        'language': 'ko'  # 결과를 한국어로 받기 위해 설정
    }
    
    if keyword:
        params['keyword'] = keyword
        
    try:
        response = requests.get(BASE_URL, params=params)
        response.raise_for_status() # HTTP 오류가 발생하면 예외 발생
        data = response.json()
        
        places_list = []
        if data.get('status') == 'OK':
            for place in data.get('results', []):
                # 필요한 핵심 정보만 추출
                lat = place['geometry']['location']['lat']
                lng = place['geometry']['location']['lng']
                
                places_list.append({
                    'name': place.get('name'),
                    'address': place.get('vicinity', '주소 정보 없음'),
                    'latitude': lat,
                    'longitude': lng,
                    'place_id': place.get('place_id'),
                    'rating': place.get('rating', 'N/A')
                })
        
        return places_list

    except requests.RequestException as e:
        print(f"API 요청 오류 발생: {e}")
        return []
    except Exception as e:
        print(f"처리 중 오류 발생: {e}")
        return []

In [2]:
# 예시 위치 (서울특별시청 근처)
target_lat = 37.5665
target_lng = 126.9780
search_radius = 1000 # 1km

# 주변 맛집 검색
restaurants = nearby_search_places(
    api_key=API_KEY,
    latitude=target_lat,
    longitude=target_lng,
    radius=search_radius,
    place_type='restaurant'
)

print(f"**📍 주변 맛집 ({len(restaurants)}개 검색됨):**")

for i, place in enumerate(restaurants[:5]): # 상위 5개만 출력
    print(f"{i+1}. {place['name']} - 평점: {place['rating']} - 주소: {place['address']}")

**📍 주변 맛집 (0개 검색됨):**


In [3]:
# 주변 관광지 검색
attractions = nearby_search_places(
    api_key=API_KEY,
    latitude=target_lat,
    longitude=target_lng,
    radius=search_radius,
    place_type='tourist_attraction'
)

print(f"\n**🌟 주변 관광지 ({len(attractions)}개 검색됨):**")

for i, place in enumerate(attractions[:5]): # 상위 5개만 출력
    print(f"{i+1}. {place['name']} - 평점: {place['rating']} - 주소: {place['address']}")


**🌟 주변 관광지 (0개 검색됨):**
